In [424]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [425]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [426]:
df = pd.read_csv(f'artifacts/temp_data/with_year_data.csv')

In [427]:
df.isnull().sum()

Age               0
Gender            0
Location          0
Monthly_Bill      0
Total_Usage_GB    0
Churn             0
year              0
month             0
dtype: int64

In [428]:
encoder = LabelEncoder()

In [429]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})
df['Location'] = encoder.fit_transform(df[['Location']])

In [430]:
X = df.drop(columns='Churn')
y = df['Churn']

In [431]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (OneHotEncoder,
                                   MinMaxScaler)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [432]:
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.40, random_state=42)

In [433]:
encoder = LabelEncoder()

In [434]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})
# df['Location'] = encoder.fit_transform(df[['Location']])
# df['Age'] = np.log(df['Age'])
# df['Subscription_Length_Months'] = np.log(df['Subscription_Length_Months'])
# df['Monthly_Bill'] = np.log(df['Monthly_Bill'])
# df['Total_Usage_GB'] = np.log(df['Total_Usage_GB'])

In [435]:
X_train

,Age,Gender,Location,Monthly_Bill,Total_Usage_GB,year,month
40507,68,1,2,71.78,483,1,6
72707,70,1,1,95.25,378,1,4
90912,50,1,1,51.44,499,1,3
28532,25,1,0,72.91,433,1,9
13006,53,1,4,97.73,448,1,1
...,...,...,...,...,...,...,...
6265,35,0,3,67.33,235,1,9
54886,56,0,0,85.40,347,1,1
76820,69,0,1,76.24,321,0,2
860,55,0,0,89.19,315,1,0


In [436]:
preprocessing = ColumnTransformer(transformers=[
    ('OHE',OneHotEncoder(drop='first',sparse=False,dtype=np.int64),['Location']),
],remainder='passthrough')

In [437]:
# df['Age'] = np.log(df['Age'])
# df['Subscription_Length_Months'] = np.log(df['Subscription_Length_Months'])
df['Monthly_Bill'] = np.log(df['Monthly_Bill'])
df['Total_Usage_GB'] = np.log(df['Total_Usage_GB'])

In [438]:
X_train= preprocessing.fit_transform(X_train)
X_test = preprocessing.transform(X_test)

## model

In [439]:
from keras.layers import BatchNormalization, Dropout, Bidirectional, LSTM, Embedding, Dense ,Input
from keras.losses import binary_crossentropy
from tensorflow import keras
from keras.callbacks import LearningRateScheduler , EarlyStopping
from keras.activations import relu , sigmoid
from keras import Sequential
from keras.initializers import he_normal

In [440]:
model = Sequential()

model.add(layer=Dense(units=512,activation=relu,kernel_initializer=he_normal))
model.add(layer=Dense(units=332,activation=relu,kernel_initializer=he_normal))
model.add(BatchNormalization())
model.add(Dense(units=128,activation=relu,kernel_initializer=he_normal))
model.add(Dense(units=64,activation=relu,kernel_initializer=he_normal))
model.add(Dense(units=1,activation=sigmoid,name='output_layer'))

In [441]:
def lr_schedule(epoch, lr):
    if epoch < 1:
        return lr
    else:
        return lr * np.exp(-0.1)

# learning rate scheduler callback to descrese the learning rate gradually as the epochs increases So that my alogrithm could not jump out of Global minima.
lr_scheduler = LearningRateScheduler(lr_schedule)

# Early stopping to stop the Neural Network when we get same Validation accuracy
# early_stopping = EarlyStopping(
#     monitor="accuracy",
#     min_delta=0.00001,
#     patience=5,
#     verbose=1,
#     mode="auto",
#     baseline=None,
#     restore_best_weights=False
# )

In [442]:
optimizer = keras.optimizers.Adam(learning_rate=1)

In [443]:
model.compile(optimizer=optimizer, 
               loss=binary_crossentropy, 
                 metrics=['accuracy']) 

In [444]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20,
                    batch_size=64,
                      callbacks=[lr_scheduler]) 

Epoch 1/20
938/938 [==============================] - 13s 10ms/step - loss: 280.7237 - accuracy: 0.5009 - val_loss: 0.7005 - val_accuracy: 0.5027 - lr: 1.0000
Epoch 2/20
938/938 [==============================] - 6s 7ms/step - loss: 0.7005 - accuracy: 0.5008 - val_loss: 0.6934 - val_accuracy: 0.5027 - lr: 0.9048
Epoch 3/20
938/938 [==============================] - 6s 7ms/step - loss: 0.7017 - accuracy: 0.5016 - val_loss: 0.7367 - val_accuracy: 0.5027 - lr: 0.8187
Epoch 4/20
938/938 [==============================] - 6s 6ms/step - loss: 0.7019 - accuracy: 0.5009 - val_loss: 0.7158 - val_accuracy: 0.4973 - lr: 0.7408
Epoch 5/20
938/938 [==============================] - 7s 8ms/step - loss: 0.7012 - accuracy: 0.4969 - val_loss: 0.6950 - val_accuracy: 0.4973 - lr: 0.6703
Epoch 6/20
938/938 [==============================] - 6s 7ms/step - loss: 0.6989 - accuracy: 0.5027 - val_loss: 0.6935 - val_accuracy: 0.5027 - lr: 0.6065
Epoch 7/20
938/938 [==============================] - 6s 6ms/step 